# Qualaria

This code is the first part of the Qualaria ligand vetting flow. This first part focuses on identifying pockets on the PfEMP1 protein and generating SMILES of potential ligands to the pockets.

We ran this code with Python 3.11.13, on Google Colab and Kaggle (as a guide for which Python version to use). It could run with Python versions below that, but we did not test it.)

In [3]:
# !pip install ipython-autotime --quiet
# %load_ext autotime

In [4]:
!pip install Bio py3Dmol
# !pip install gdown

In [5]:
# !pip install rdkit rdkit-pypi gdown biopython pyyaml easydict tensorboard lmdb prody pypdb

In [6]:
!pip uninstall numpy -y
!pip install numpy==1.26.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have googl

In [7]:
!pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric
!pip install torch==2.1.0 --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 446.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 19.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires to

In [8]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.3 MB/s eta 0:00:0000:010:01


In [9]:
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch-geometric==2.3.1

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 12.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910496 sha256=a42e74078fe517e50b0871062b611d4789a1577be6e60e47cd507b64a0eb2fb5
  Stored in directory: /root/.cache/pip/wheels/25/78/6c/9fd091ca1c5e137c66cbd03696ffa14f75e9abc5abfe0dbcc6
Successfully built torch-geometric


In [10]:
import os
import subprocess
import requests
import numpy as np
import py3Dmol
import torch

from Bio.PDB.PDBParser import PDBParser

## 1. Identify PfEMP1's pockets (functional sites)

In [11]:
class FPocketPocketFinder:
    def __init__(self, pdb_file, pocket_visualizer=None, base_dir=None, toolkit_dir=None, image_dir=None):
        self.pdb_file = pdb_file # Protein file
        self.base_dir = base_dir or os.getcwd()
        self.toolkit_dir = toolkit_dir or os.getcwd()
        self.pocket_visualizer = pocket_visualizer
        if self.pocket_visualizer:
            self.pocket_visualizer.load_main_protein_structure(self.pdb_file)

        self.image_dir = image_dir

        self.pdb_prefix = os.path.splitext(os.path.basename(self.pdb_file))[0]
        self.pockets_data_dir = os.path.join(self.base_dir, self.pdb_prefix+"_out")
        self.pockets_dir = os.path.join(self.pockets_data_dir, "pockets")
        self.pockets_info_path = os.path.join(self.pockets_data_dir, self.pdb_prefix+"_info.txt")

        self.fpocket_path = os.path.join(self.toolkit_dir, "fpocket")
        self.fpocket_bin_folder = os.path.join(self.fpocket_path, "bin")
        self.fpocket_bin = os.path.join(self.fpocket_bin_folder, "fpocket")
        self.install_fpocket()

        self.pockets = {}
        self.pocket_data = {}


    def install_fpocket(self):
        """Clones and builds fpocket from GitHub."""

        if os.path.isdir(self.fpocket_path):
            print("fpocket already cloned.")
        else:
            print("Cloning fpocket repository...")
            command = ["git", "clone",
                       "--branch", "4.2.2",
                       "--depth", "1",
                       "https://github.com/Discngine/fpocket.git",]
            subprocess.run(command, cwd=self.toolkit_dir, check=True)

        print("Building fpocket...")
        subprocess.run(["make"], cwd=self.fpocket_path, check=True)

        ## Add fpocket binary to PATH
        ## fpocket_bin_folder = os.path.abspath(f"{fpocket_path}/bin")
        # os.environ["PATH"] += os.pathsep + self.fpocket_bin_folder
        # print(f"fpocket installed and added to PATH: {self.fpocket_bin_folder}")


    def find_pockets(self, force_pocket_regeneration=False):
        """Runs fpocket on the given PDB file."""

        pdb_file = self.pdb_file

        if os.path.isfile(self.pockets_info_path) and not force_pocket_regeneration:
            print(f"Pockets already generated for pdb file: {pdb_file}.\nTo force regeneration, consider setting the argument `force_pocket_regeneration` to True.")
            return self.list_pockets()

        print(f"Running fpocket on {pdb_file}...")
        command = [self.fpocket_bin, "-f", pdb_file]
        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode == 0:
            print("fpocket finished successfully.")
            return self.list_pockets()
        else:
            print("fpocket failed:")
            print(result.stderr)
            return []


    def list_pockets(self):
        """Lists the pockets found for the given PDB file."""

        pockets_dir = self.pockets_dir

        if not os.path.exists(pockets_dir):
            print("Pocket directory not found.")
            return []

        self.pockets = sorted(
            [f for f in os.listdir(pockets_dir) if f.endswith('_atm.pdb')])

        return self.pockets


    def get_pocket_center(self, pocket_pdb_path):
        """Returns the center of the given pocket."""

        parser = PDBParser(QUIET=True)
        structure = parser.get_structure("pocket", pocket_pdb_path)

        if not structure:
            raise ValueError("No structure found in pocket file.")

        coords = []
        for atom in structure.get_atoms():
            coords.append(atom.coord)

        if not coords:
            raise ValueError("No atoms found in pocket file.")

        center = np.mean(coords, axis=0)
        return tuple(center)


    def get_pocket_residues(self, pocket_pdb_path):
        """Returns the residues in the given pocket."""

        parser = PDBParser(QUIET=True)
        structure = parser.get_structure("pocket", pocket_pdb_path)

        if not structure:
            raise ValueError("No structure found in pocket file.")

        residues = []
        for model in structure:
            for chain in model:
                for residue in chain:
                    res_id = residue.get_id()
                    res_name = residue.get_resname()
                    residues.append((res_name, res_id[1]))
        return residues


    def get_all_pockets_data(self):
        """Returns the data for all pockets."""

        pockets_dir = self.pockets_dir

        if not os.path.exists(pockets_dir):
            raise FileNotFoundError(f"Directory not found: {pockets_dir}")

        pocket_data = {}

        for pocket_file in self.pockets:
            pocket_path = os.path.join(pockets_dir, pocket_file)
            with open(pocket_path, 'r') as f:
                pdb_content = f.read()
                pocket_data[pocket_file] = pdb_content

        self.pocket_data = pocket_data

        return pocket_data


    def visualize_all_pockets(self):
        """Visualizes all pockets."""

        if not self.pocket_visualizer:
            raise ValueError("PocketVisualizer not set.")

        for i, (pocket_name, pocket_pdb) in enumerate(self.pocket_data.items()):
            print(f"Pocket {i+1}: {pocket_name}")
            self.pocket_visualizer.visualize_pocket(pocket_pdb)


    def save_all_pockets_image_as_html(self):
        """Visualizes all pockets."""

        if not self.pocket_visualizer:
            raise ValueError("PocketVisualizer not set.")

        image_dir = self.image_dir

        for i, (pocket_name, pocket_pdb) in enumerate(self.pocket_data.items()):
            view = self.pocket_visualizer.build_pocket_view(pocket_pdb)
            image_name = pocket_name.replace(".pdb", ".html")
            image_path = os.path.join(image_dir, image_name)

            self.pocket_visualizer.save_pocket_3d_image_to_html(view, image_path)

In [12]:
class Py3DMolPocketVisualizer:
    def __init__(self, pdb_file=None):
        self.pdb_file = pdb_file


    def load_main_protein_structure(self, pdb_file):
        """Loads the protein structure from the given PDB file."""

        self.pdb_file = pdb_file


    def build_pocket_view(self, pocket_pdb):
        """Builds a py3Dmol view for the given pocket."""

        if not self.pdb_file:
            raise ValueError("PDB file not set.")

        with open(self.pdb_file, 'r') as f:
            protein_pdb = f.read()

        view = py3Dmol.view(width=500, height=300)

        view.addModel(protein_pdb, 'pdb')
        view.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum'}})

        view.addModel(pocket_pdb, 'pdb')
        view.setStyle({'model': 1}, {'stick': {'colorscheme': 'orangeCarbon'}})
        view.addSurface(py3Dmol.VDW, {'opacity': 0.8, 'color': 'orange'}, {'model': 1})

        return view


    def visualize_pocket(self, pocket_pdb):
        """Visualizes a single pocket."""

        view = self.build_pocket_view(pocket_pdb)

        view.zoomTo()
        view.show()


    def save_pocket_3d_image_to_html(self, view, output_file='structure.html'):
        html_str = view._make_html()
        with open(output_file, 'w') as f:
            f.write(html_str)

In [13]:
class AlphaFoldService:
    def download_protein_structure(self, uniprot_id, output_path, force_download=False):
        if os.path.isfile(output_path) and not force_download:
            print(f"AlphaFold structure already downloaded at path: {output_path}.\nTo force download, consider setting the argument `force_download` to True.")
            return

        url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
        response = requests.get(url)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded AlphaFold structure: {output_path}")
        else:
            raise Exception("Failed to download structure: " + response.reason)

In [14]:
base_dir = "/kaggle/working/" # Modify this according to the environment (Colab, Kaggle, Personal Jupyter Notebook, ...)

In [15]:
output_dir = os.path.join(base_dir, "output")
os.makedirs(output_dir, exist_ok=True)

toolkit_dir = os.path.join(base_dir, "toolkit")
os.makedirs(toolkit_dir, exist_ok=True)

config_dir = os.path.join(base_dir, "config")
os.makedirs(config_dir, exist_ok=True)

In [16]:
# Search protein id on AlphaFold's website: https://alphafold.ebi.ac.uk
uniprot_id = "Q8I3Y6"

In [17]:
pdb_file_name = "PfEMP1.pdb"

In [18]:
# Create folder for the specific protein. This folder will have all the
# generated output related to this protein: protein file, pockets, ligands.
pdb_prefix = os.path.splitext(os.path.basename(pdb_file_name))[0]
pdb_dir = os.path.join(output_dir, pdb_prefix)
os.makedirs(pdb_dir, exist_ok=True)
pdb_file = os.path.join(pdb_dir, pdb_file_name)

image_dir = os.path.join(pdb_dir, "images")
os.makedirs(image_dir, exist_ok=True)

ligands_dir = os.path.join(pdb_dir, "ligands")
os.makedirs(ligands_dir, exist_ok=True)

In [19]:
alpha_fold = AlphaFoldService()
alpha_fold.download_protein_structure(uniprot_id, pdb_file)

AlphaFold structure already downloaded at path: /kaggle/working/output/PfEMP1/PfEMP1.pdb.
To force download, consider setting the argument `force_download` to True.


In [20]:
pocket_visualizer = Py3DMolPocketVisualizer()

In [21]:
pocket_finder = FPocketPocketFinder(pdb_file, pocket_visualizer, pdb_dir, toolkit_dir, image_dir)

fpocket already cloned.
Building fpocket...
cd src/qhull/ && make
make[1]: Entering directory '/kaggle/working/toolkit/fpocket/src/qhull'
mkdir -p bin lib
==== If ar fails, try make qhullx ====
ar -rs lib/libqhullstatic.a src/libqhullstatic/global.o src/libqhullstatic/stat.o src/libqhullstatic/geom2.o src/libqhullstatic/poly2.o src/libqhullstatic/merge.o src/libqhullstatic/libqhull.o src/libqhullstatic/geom.o src/libqhullstatic/poly.o src/libqhullstatic/qset.o src/libqhullstatic/mem.o src/libqhullstatic/random.o src/libqhullstatic/usermem.o src/libqhullstatic/userprintf.o src/libqhullstatic/io.o src/libqhullstatic/user.o src/libqhullstatic/rboxlib.o src/libqhullstatic/userprintf_rbox.o
#If 'ar -rs' fails try using 'ar -s' with 'ranlib'
#ranlib lib/libqhullstatic.a
gcc -o bin/rbox src/rbox/rbox.o -O3 -ansi -Isrc -fpic -Wall -Wcast-qual -Wextra -Wwrite-strings -Wshadow -Llib -lqhullstatic -lm
gcc -o bin/qdelaunay src/qdelaunay/qdelaun.o -O3 -ansi -Isrc -fpic -Wall -Wcast-qual -Wextra -Ww

src/fpmain.c: In function ‘open_file_format’:
src/fpmain.c:239:16: warning: ‘pdb’ may be used uninitialized in this function [-Wmaybe-uninitialized]
  239 |         return pdb;
      |                ^~~


gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/psorting.c -o obj/psorting.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/pscoring.c -o obj/pscoring.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/utils.c -o obj/utils.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/pertable.c -o obj/pertable.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/memhandler.c -o obj/memhandler.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEB

src/writepdb.c: In function ‘write_pdb_atom_line’:
src/writepdb.c:94:18: warning: comparison between pointer and integer
   94 |         if(insert!=NULL) {
      |                  ^~
src/writepdb.c:101:34: warning: ‘%05x’ directive writing between 5 and 8 bytes into a region of size 6 [-Wformat-overflow=]
  101 |                 sprintf(id_buf, "%05x", id);
      |                                  ^~~~
src/writepdb.c:101:33: note: directive argument in the range [100000, 2147483647]
  101 |                 sprintf(id_buf, "%05x", id);
      |                                 ^~~~~~
In file included from /usr/include/stdio.h:894,
                 from src/../headers/writepdb.h:14,
                 from src/writepdb.c:2:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:38:10: note: ‘__builtin___sprintf_chk’ output between 6 and 9 bytes into a destination of size 6
   38 |   return __builtin___sprintf_chk (__s, __USE_FORTIFY_LEVEL - 1,
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/rpdb.c -o obj/rpdb.o


src/rpdb.c: In function ‘rpdb_open’:
src/rpdb.c:902:42: warning: ‘z’ may be used uninitialized in this function [-Wmaybe-uninitialized]
  902 |                 if (x < 9990 && y < 9990 && z < 9990)
      |                     ~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~
src/rpdb.c:902:30: warning: ‘y’ may be used uninitialized in this function [-Wmaybe-uninitialized]
  902 |                 if (x < 9990 && y < 9990 && z < 9990)
      |                     ~~~~~~~~~^~~~~~~~~~~
src/rpdb.c:902:20: warning: ‘x’ may be used uninitialized in this function [-Wmaybe-uninitialized]
  902 |                 if (x < 9990 && y < 9990 && z < 9990)
      |                    ^
src/rpdb.c: In function ‘rpdb_read’:
src/rpdb.c:1234:48: warning: ‘tmpz’ may be used uninitialized in this function [-Wmaybe-uninitialized]
 1234 |                 if (tmpx < 9990 && tmpy < 9990 && tmpz < 9990)
      |                     ~~~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~
src/rpdb.c:1234:33: warning: ‘tmpy’ may be used uninitialize

gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/tparams.c -o obj/tparams.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/fparams.c -o obj/fparams.o


src/fparams.c: In function ‘get_fpocket_args’:
src/fparams.c:296:24: warning: passing argument 1 of ‘strcpy’ from incompatible pointer type [-Wincompatible-pointer-types]
  296 |                 strcpy(&residue_string, pt);
      |                        ^~~~~~~~~~~~~~~
      |                        |
      |                        char * (*)[8000]
In file included from /usr/include/features.h:486,
                 from /usr/include/x86_64-linux-gnu/bits/libc-header-start.h:33,
                 from /usr/include/stdio.h:27,
                 from src/../headers/fparams.h:16,
                 from src/fparams.c:2:
/usr/include/x86_64-linux-gnu/bits/string_fortified.h:77:1: note: expected ‘char * restrict’ but argument is of type ‘char * (*)[8000]’
   77 | __NTH (strcpy (char *__restrict __dest, const char *__restrict __src))
      | ^~~~~
src/fparams.c:297:23: warning: assignment to ‘char *’ from incompatible pointer type ‘char **’ [-Wincompatible-pointer-types]
  297 |                 

gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/pocket.c -o obj/pocket.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/refine.c -o obj/refine.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/descriptors.c -o obj/descriptors.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/aa.c -o obj/aa.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg -std=gnu99 -Iplugins/include -Iplugins/LINUXAMD64/molfile -c src/fpocket.c -o obj/fpocket.o
gcc -W -Wextra -Wwrite-strings -Wstrict-prototypes -DM_OS_LINUX -DMNO_MEM_DEBUG -O2 -g -pg 

In [22]:
pockets = pocket_finder.find_pockets()

Pockets already generated for pdb file: /kaggle/working/output/PfEMP1/PfEMP1.pdb.
To force regeneration, consider setting the argument `force_pocket_regeneration` to True.


In [23]:
print(f"Found {len(pockets)} pockets.")

for index, pocket in enumerate(pockets):
    print(f"Pocket {index+1}: {pocket}")

Found 4 pockets.
Pocket 1: pocket1_atm.pdb
Pocket 2: pocket2_atm.pdb
Pocket 3: pocket3_atm.pdb
Pocket 4: pocket4_atm.pdb


In [24]:
pockets_data = pocket_finder.get_all_pockets_data()

In [25]:
pocket_centers = {}

for pocket_file in pockets:
    pocket_full_path = os.path.join(pocket_finder.pockets_dir, pocket_file)
    center = pocket_finder.get_pocket_center(pocket_full_path)
    print(f"Pocket {pocket_file} center: {center}")
    pocket_centers[pocket_file] = center

Pocket pocket1_atm.pdb center: (-18.919567, -2.989, 13.69187)
Pocket pocket2_atm.pdb center: (10.693321, 2.3687997, -23.15884)
Pocket pocket3_atm.pdb center: (12.2715225, 3.3361301, -13.966915)
Pocket pocket4_atm.pdb center: (18.172617, 3.806857, -20.537235)


In [26]:
for pocket_file in pockets:
    pocket_full_path = os.path.join(pocket_finder.pockets_dir, pocket_file)
    residues = pocket_finder.get_pocket_residues(pocket_full_path)
    print(f"Pocket {pocket_file} has {len(residues)} residues. Sample:", residues)

Pocket pocket1_atm.pdb has 7 residues. Sample: [('LYS', 97), ('LEU', 101), ('LYS', 100), ('ALA', 15), ('LEU', 16), ('SER', 19), ('ASP', 12)]
Pocket pocket2_atm.pdb has 6 residues. Sample: [('SER', 67), ('ASP', 49), ('GLU', 65), ('LYS', 66), ('TYR', 70), ('ASP', 71)]
Pocket pocket3_atm.pdb has 9 residues. Sample: [('LEU', 40), ('HIS', 54), ('CYS', 69), ('LEU', 53), ('GLU', 64), ('LYS', 55), ('VAL', 44), ('THR', 72), ('ILE', 73)]
Pocket pocket4_atm.pdb has 8 residues. Sample: [('LYS', 63), ('GLU', 65), ('GLU', 64), ('ASP', 49), ('SER', 51), ('ASP', 50), ('VAL', 52), ('LEU', 53)]


In [27]:
pocket_finder.visualize_all_pockets()

Pocket 1: pocket1_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 2: pocket2_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 3: pocket3_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 4: pocket4_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [28]:
pocket_finder.save_all_pockets_image_as_html()

## 2. Generate ligands

**Note:** In this notebook, we use the terms "SMILES" and "ligand" interchangeably, but these two terms are technically different.

- A SMILES string refers to the textual representation of a molecule in general.
- A ligand is an actual molecule that can bind to a protein.

When we say we generate ligands, what we actually do is to generate the SMILES strings of these ligands. These textual representations can then be used to reconstruct the actual molecules (ligands) in the lab.

Since our focus here is to generate the ligands' molecule in the form of SMILES strings, we allow ourselves to use these terms as if they are the same. So, anywhere you see SMILES, in this code, think "SMILES of a ligand". And, when you see ligand, think "the text representation of that ligand", so, still the "SMILES of that ligand".

For reference, the term SMILES stands for "Simplified Molecular Input Line Entry System".

#### Install Pocket2Mol

In [29]:
# # Download checkpoint
# !gdown 1sK4LjgDNZ0TWK1gRjFePYGi9m9oy6iyd -O ckpt/pocket2mol.pth

This is the custom config for Pocket2Mol. Save this in a file named `pocket2mol_config.yml` and upload it to Colab or add it to the Jupyter Notebook.

```yml
model:
    checkpoint: ./ckpt/pretrained_Pocket2Mol.pt

sample:
  seed: 2020
  num_samples: 10 # Adjust the number of samples to be generated. A big number will also increase the time of generation.
  beam_size: 300
  max_steps: 50
  threshold:
    focal_threshold: 0.5
    pos_threshold: 0.25
    element_threshold: 0.3
    hasatom_threshold: 0.6
    bond_threshold: 0.4
```

In [30]:
!pip install rdkit
# !pip install pyyaml
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 44.7 MB/s eta 0:00:00:00:0100:01


In [31]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 7.0 MB/s eta 0:00:00:00:01


In [32]:
import yaml
import os
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, QED

class Pocket2MolLigandGenerator:
    def __init__(self, pdb_file, output_dir=None, config_dir=None, toolkit_dir=None, ligands_dir=None):
        self.pdb_file = pdb_file
        self.toolkit_dir = toolkit_dir or os.getcwd()
        self.output_dir = output_dir or os.getcwd()
        self.config_dir = config_dir or os.getcwd()
        self.ligands_dir = ligands_dir or os.getcwd()

        pocket2mol_config = {
            "model": {"checkpoint": "./ckpt/pretrained_Pocket2Mol.pt"},
            "sample":
                {"seed": 2020,
                 "num_samples": 10, # Adjust the number of samples to be generated. A big number will also increase the time of generation.
                 "beam_size": 300,
                 "max_steps": 50,
                 "threshold": {"focal_threshold": 0.5, "pos_threshold": 0.25,
                               "element_threshold": 0.3, "hasatom_threshold": 0.6,
                               "bond_threshold": 0.4}
                }
        }

        self.config_path = os.path.join(self.config_dir, "pocket2mol_config.yml")

        with open(self.config_path, "w") as f:
            yaml.dump(pocket2mol_config, f, default_flow_style=False, sort_keys=False)

        self.pocket2mol_path = os.path.join(self.toolkit_dir, "Pocket2Mol")
        self.pocket2mol_model_dir = os.path.join(self.pocket2mol_path, "ckpt")
        self.pocket2mol_model = os.path.join(self.pocket2mol_model_dir, "pretrained_Pocket2Mol.pt")

        self.install_pocket2mol()


    def install_pocket2mol(self):
        """Clones and builds Pocket2Mol from GitHub."""

        # Pocket2Mol repo does not have any tag, so we clone based on a commit
        # hash. This is important so that, if the package is updated with
        # breaking changes, it does not affect this code.
        repo_url = "https://github.com/pengxingang/Pocket2Mol.git"
        commit_hash = "836a0c4ce487297ad24bc54ac2ebd163de13242c"
        repo_path = self.pocket2mol_path

        if os.path.isdir(repo_path):
            print("Pocket2Mol already cloned.")
            return

        print("Cloning Pocket2Mol repo...")
        subprocess.run(["git", "clone", "--depth", "1", repo_url], cwd=self.toolkit_dir, check=True)

        print(f"Fetching and checking out commit {commit_hash}...")
        subprocess.run(["git", "fetch", "--depth", "1", "origin", commit_hash], cwd=repo_path, check=True)
        subprocess.run(["git", "checkout", commit_hash], cwd=repo_path, check=True)

        gdrive_id = "1WaoEj9RDG4VEcyHEmgsjbh958txm1W6x"

        if not os.path.isfile(self.pocket2mol_model):
            print(f"Downloading model file {self.pocket2mol_model}...")
            subprocess.run(["gdown", gdrive_id], cwd=self.pocket2mol_model_dir, check=True)
        else:
            print(f"Model file already exists: {self.pocket2mol_model}")

        print(f"Pocket2Mol installed.")


    def generate_ligands(self, pocket_center, pocket_name, force_ligand_regeneration=False):
        """Generates ligands for the given pocket center."""

        pocket_prefix = os.path.splitext(pocket_name)[0]
        smiles_output_dir = os.path.join(self.ligands_dir, pocket_prefix)

        if not os.path.isdir(smiles_output_dir) or force_ligand_regeneration:
            print(f"\nGenerating ligands for pocket: {pocket_name}")
            os.makedirs(smiles_output_dir, exist_ok=True)
            self.generate_smiles(pocket_center, smiles_output_dir)
        else:
            print(f"\nLigands already generated for pocket: {pocket_name}.\nTo force regeneration, consider setting the argument `force_ligand_regeneration` to True.")
        
        smiles_path = self.get_latest_smiles_file(smiles_output_dir)
        smiles = self.filter_smiles(smiles_path)

        ligands = []

        for index in range(len(smiles)):
            smi, qed, mw = smiles[index]
            ligand_path = os.path.join(smiles_output_dir, f"{index+1}-smi-{smi}.sdf")
            self.smiles_to_sdf(smi, ligand_path)
            coords = self.compute_molecule_coordinates(ligand_path)
            path_to_record = os.path.relpath(
                ligand_path, start=os.path.dirname(self.output_dir),
            )

            ligands.append({
                "smiles": smi,
                "qed": qed,
                "mw": mw,
                "path": path_to_record,
                "coords": coords,
            })

        return ligands


    def generate_smiles(self, pocket_center, output_dir):
        """Generates SMILES for the given pocket center."""

        center_str = ",".join(f"{c:.2f}" for c in pocket_center)
        print(f"Pocket center: {center_str}")

        command = [
            "python", os.path.join(self.pocket2mol_path, "sample_for_pdb.py"),
            "--pdb_path", self.pdb_file,
            "--center", " " + center_str,
            "--outdir", output_dir,
            "--config", self.config_path
        ]

        print()
        print("Running Pocket2Mol...")
        result = subprocess.run(command, cwd=self.pocket2mol_path, capture_output=True, text=True)

        if result.returncode == 0:
            print("Pocket2Mol completed successfully.")
            return result
        else:
            print("Pocket2Mol failed:")
            print(result.stderr)


    def get_latest_smiles_file(self, output_dir):
        """Find the newest timestamped directory."""

        subdirs = [
            os.path.join(output_dir, d)
            for d in os.listdir(output_dir)
            if os.path.isdir(os.path.join(output_dir, d))
        ]

        if not subdirs:
            raise FileNotFoundError(f"No output folders found in {output_dir}")

        latest_dir = max(subdirs, key=os.path.getmtime)
        smiles_path = os.path.join(latest_dir, "SMILES.txt")

        if not os.path.isfile(smiles_path):
            raise FileNotFoundError(f"SMILES.txt not found in {latest_dir}")

        return smiles_path


    def filter_smiles(self, file_path, top_n=5):
        """Filters SMILES based on QED, MW, and Lipinski rules."""

        with open(file_path) as f:
            smiles_list = [line.strip() for line in f if line.strip()]

        filtered = []

        for smi in smiles_list:
            mol = Chem.MolFromSmiles(smi)
            if not mol:
                continue

            mw = Descriptors.MolWt(mol)
            qed = QED.qed(mol)
            hba = Descriptors.NumHAcceptors(mol)
            hbd = Descriptors.NumHDonors(mol)
            logp = Descriptors.MolLogP(mol)

            lipinski = (hba <= 10) and (hbd <= 5) and (logp <= 5)
            if 100 < mw <= 600 and qed >= 0.4 and lipinski:
                filtered.append((smi, qed, mw))

        # Sort by QED score
        filtered.sort(key=lambda x: x[1], reverse=True)

        top_filtered = filtered[:top_n]
        print(f"\nSelected top {len(top_filtered)} ligands:")
        for smi, qed, mw in top_filtered:
            print(f"{smi} | QED: {qed:.2f} | MW: {mw:.2f}")

        return top_filtered


    def smiles_to_sdf(self, smiles, sdf_path="molecule.sdf"):
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)  # Add hydrogens
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Generate 3D coordinates
        AllChem.UFFOptimizeMolecule(mol)  # Optional: minimize energy
        writer = Chem.SDWriter(sdf_path)
        writer.write(mol)
        writer.close()

        relative_sdf_path = os.path.relpath(sdf_path, start=os.path.dirname(self.output_dir))
        print(f"Saved SDF to: {relative_sdf_path}")

        return sdf_path


    def compute_molecule_coordinates(self, sdf_path):
        # Load the molecule from an SDF file
        supplier = Chem.SDMolSupplier(sdf_path, removeHs=False)
        mol = supplier[0]  # First molecule in the file

        if mol is None:
            raise ValueError("Could not read molecule from SDF. Check the file format.")

        # Generate 3D coordinates if not already present
        if not mol.GetConformer().Is3D():
            AllChem.EmbedMolecule(mol, AllChem.ETKDG())
            AllChem.MMFFOptimizeMolecule(mol)

        # Extract coordinates and format them
        conf = mol.GetConformer()
        coords = []

        for i in range(mol.GetNumAtoms()):
            atom = mol.GetAtomWithIdx(i)
            pos = conf.GetAtomPosition(i)
            coords.append(f"{atom.GetSymbol()} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}")

        formatted_coords = "; ".join(coords)

        return formatted_coords


In [33]:
ligand_generator = Pocket2MolLigandGenerator(
    pdb_file=pdb_file,
    output_dir=output_dir,
    config_dir=config_dir,
    toolkit_dir=toolkit_dir,
    ligands_dir=ligands_dir
)

Pocket2Mol already cloned.


In [34]:
import json

all_ligands = {}

for pocket_file_name, center in pocket_centers.items():
    ligands = ligand_generator.generate_ligands(center, pocket_file_name)
    all_ligands[pocket_file_name] = ligands


Ligands already generated for pocket: pocket1_atm.pdb.
To force regeneration, consider setting the argument `force_ligand_regeneration` to True.

Selected top 5 ligands:
O=C(O)CCc1cccc(CO)c1 | QED: 0.73 | MW: 180.20
COc1ccc2c(O)cccc2c1 | QED: 0.72 | MW: 174.20
COC1CCc2c(O)cccc2C1 | QED: 0.71 | MW: 178.23
O=C(O)COc1ccccc1 | QED: 0.71 | MW: 152.15
O=C(O)c1cccc(C(=O)O)c1 | QED: 0.69 | MW: 166.13
Saved SDF to: output/PfEMP1/ligands/pocket1_atm/1-smi-O=C(O)CCc1cccc(CO)c1.sdf
Saved SDF to: output/PfEMP1/ligands/pocket1_atm/2-smi-COc1ccc2c(O)cccc2c1.sdf
Saved SDF to: output/PfEMP1/ligands/pocket1_atm/3-smi-COC1CCc2c(O)cccc2C1.sdf
Saved SDF to: output/PfEMP1/ligands/pocket1_atm/4-smi-O=C(O)COc1ccccc1.sdf
Saved SDF to: output/PfEMP1/ligands/pocket1_atm/5-smi-O=C(O)c1cccc(C(=O)O)c1.sdf

Ligands already generated for pocket: pocket2_atm.pdb.
To force regeneration, consider setting the argument `force_ligand_regeneration` to True.

Selected top 5 ligands:
O=C(O)CCc1ccccc1 | QED: 0.71 | MW: 150.18

In [35]:
json_path = os.path.join(pdb_dir, "ligands.json")

with open(json_path, "w") as f:
    json.dump(all_ligands, f)

print(f"Ligands coordinates and data saved to: {json_path}.")

Ligands coordinates and data saved to: /kaggle/working/output/PfEMP1/ligands.json.


### Save essential files for energy calculations in part 2 (protein, pockets, ligands, protein-ligands dockings)

In [36]:
import zipfile

generated_files_to_download_prefix = f"{pdb_prefix}_pockets_ligands"
generated_files_to_download = f"{generated_files_to_download_prefix}.zip"
generated_files_to_download_path = os.path.join(pdb_dir, generated_files_to_download)

with zipfile.ZipFile(generated_files_to_download_path, mode="w") as zipf:
    pass  # Creates the zip but adds nothing yet

with zipfile.ZipFile(generated_files_to_download_path, mode="a") as zipf:
    archive_base_dir = generated_files_to_download_prefix
    archive_pockets_dir = f"{archive_base_dir}/pockets"
    archive_ligands_dir = f"{archive_base_dir}/ligands"
    
    zipf.write(pdb_file, arcname=f"{archive_base_dir}/{pdb_file_name}")
    zipf.write(json_path, arcname=f"{archive_ligands_dir}/ligands.json")

    for pocket_name in pockets:
        pocket_full_path = os.path.join(pocket_finder.pockets_dir, pocket_name)
        pocket_path_in_archive = f"{archive_pockets_dir}/{pocket_name}"
        zipf.write(pocket_full_path, arcname=pocket_path_in_archive)

    for pocket_name, ligands in all_ligands.items():
        # We use index here to differentiate the ligands because sometimes the generated
        # ligands have the same name in lowercase but the actual name are different because the have different cases
        # And such smiles are not the same since smiles are case-sensitive.
        for ligand in ligands:
            ligand_sdf_file_path = os.path.join(base_dir, ligand['path'])
            ligand_name = os.path.basename(ligand['path'])
            pocket_name_prefix = os.path.splitext(pocket_name)[0]
            ligand_path_in_archive = f"{archive_ligands_dir}/{pocket_name_prefix}/{ligand_name}"
            zipf.write(pocket_full_path, arcname=ligand_path_in_archive)

print(f"Pockets and ligands data saved to: {os.path.relpath(generated_files_to_download_path, start=base_dir)}.\nYou can download that file and load it in the VQE notebook.")

Pockets and ligands data saved to: output/PfEMP1/PfEMP1_pockets_ligands.zip.
You can download that file and load it in the VQE notebook.


In [37]:
# import zipfile

# def zip_folder(folder_path, zip_name):
#     with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, _, files in os.walk(folder_path):
#             for file in files:
#                 full_path = os.path.join(root, file)
#                 relative_path = os.path.relpath(full_path, folder_path)
#                 zipf.write(full_path, arcname=relative_path)

In [38]:
# zip_folder(config_dir, "config.zip")

In [39]:
# zip_folder(output_dir, "output.zip")

In [40]:
# zip_folder(toolkit_dir, "toolkit.zip")

In [41]:
# !pip freeze > requirements.txt

In [42]:
# !cat requirements.txt

## 3. Evaluating the Ground State Energies

We moved this to its own notebook to avoid package conflicts.